# GraphFrames

Dans ce TP, nous allons utiliser la librairie GraphFrame. Afin de pouvoir utiliser la dernière version de celle-ci, rattacher ce notebook à un cluster 14.3 ML, qui contient les librairies compatibles.

In [ ]:
%sh /databricks/python3/bin/pip install graphframes

In [ ]:
from functools import reduce
from pyspark.sql import functions as F
from graphframes import GraphFrame

Il est possible de créer simplement des GraphFrames à partir de DataFrames de sommets et d'arêtes.

<br>- Le DataFrame de sommets doit contenir une colonne spéciale, nommée "id", qui spécifie des identifiants uniques pour chaque sommet dans le graphe.
<br>- Le DataFrame d'arêtes doit contenir deux colonnes spéciales : "src" (identifiant du sommet source de l'arête) et "dst" (identifiant du sommet destination de l'arête).
<br>- Les deux DataFrames peuvent avoir d'autres colonnes arbitraires. Ces colonnes peuvent représenter des attributs de sommets et d'arêtes.

On se donne la liste des personnes suivantes, caractérisée par 3 champs : l'id de la personne (en réalité du futur noeud), son nom et son âge. Ces personnes sont destinées à être les noeuds de notre futur graphe.

In [ ]:
people = [
    ("a", "Alice", 34),
    ("b", "Bob", 36),
    ("c", "Charlie", 30),
    ("d", "David", 29),
    ("e", "Esther", 32),
    ("f", "Fanny", 36),
    ("g", "Gabby", 60),
]

Créer un dataframe vertices à partir de cette liste.

In [ ]:
# Votre code ici

Ces personnes sont inscrites à un réseau social où deux options sont possibles :
- suivre une personne ("follow"),
- déclarer être ami avec cette personne ("friend"), ce qui n'est pas forcément réciproque.
Il n'est pas possible de cumuler les deux status.

Voici l'ensemble des relations :
<br>- Alice a déclaré être amie avec Bob.
<br>- Bob suit Charlie.
<br>- Charlie suit Bob.
<br>- Fanny suit Charlie.
<br>- Esther suit Fanny.
<br>- Esther a déclaré être amie avec David.
<br>- David a déclaré être ami avec Alice.
<br>- Alice a déclaré être amie avec Esther.

Créer un dataframe contenant l'ensemble de ces interactions. Une interaction sera caractérisée par l'identifiant de la personne à l'origine de l'interaction ("src"), celui de la personne visée par l'interaction ("dst"), et le statut de la relation ("relationship", valant "follow" ou "friend").

In [ ]:
# Votre code ici

Créer un graphe représentatif de la situation à partir de ces deux dataframes.

In [ ]:
# Votre code ici


## Requêtes de base sur les graphes et les DataFrames
Les objets de type GraphFrame fournissent plusieurs méthodes natives pour manipuler les graphes. Nous allons les manipuler ici.

Afficher les sommets/noeuds du graphe, ainsi que ses arcs. Afin de savoir dans quels attributs de notre objet ils sont stockés, consulter la documentation de la classe GraphFrame : https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html.

In [ ]:
# Votre code ici

Déterminer le "degré entrant" de l'ensemble des sommets (i.e. pour chaque sommet, le nombre de relations qui pointent vers ce sommet).

In [ ]:
# Votre code ici

Déterminer le degré sortant de l'ensemble des sommets.

In [ ]:
# Votre code ici

Déterminer le degré des sommets (somme des degrés entrant et sortant).

In [ ]:
# Votre code ici

Il est possible d'exécuter directement des requêtes sur le DataFrame des sommets.
Trouver l'âge de la personne la plus jeune dans le graphe. Utiliser le DSL.

In [ ]:
# Votre code ici

Il est bien entendu également possible d'exécuter des requêtes sur le DataFrame des arcs.
Compter le nombre de relations de type "follow" dans le graphe.

In [ ]:
# Votre code ici

## Trouver des motifs

En utilisant des motifs, il est possible construire des relations plus complexes impliquant des arêtes et des sommets.

<br>Les motifs à rechercher sont dénotés par des expressions.
<br>Un expression élémentaire est "(a)-[e]->(b)". Elle signifie qu'il existe une arête dirigée de a vers b.
<br>Il est possible de combiner ces expressions (le symbole ; est utilisé pour exprimer le ET logique).
<br>Pour les sommets, la répétition d'une lettre signifie que la référence se fait à un même sommet.
<br> Pour les arêtes, il n'est pas possible de répéter la même lettre.
<br>Une fois l'expression construite, il faut appeler la méthode find de la manière suivante : graph.find(motif)

Trouver toutes les paires de sommets avec des arêtes dans les deux directions entre eux (les relations directe et réciproque ne sont pas forcément les mêmes). Le résultat doit être un DataFrame, dans lequel les noms de colonnes sont donnés par les clés du motif.

In [ ]:
# Votre code ici

Puisque le résultat est un DataFrame, il est possible d'exécuter des requêtes par dessus le motif.
Parmi les relations précédentes, déterminer celles qui concernent deux personnes dont l'une au moins est âgée de 34 ans ou plus.

In [ ]:
# Votre code ici


## Requêtes stateful
La plupart des requêtes motif sont sans état et simples à exprimer, comme dans notre exemple précédent. Parfois, une requête plus complexe doit transporter un état le long d'un chemin dans le motif. On peut l'exprimer en combinant la recherche de motifs GraphFrame avec des filtres sur le résultat utilisant des opérations de séquence, agissant sur les colonnes du DataFrame.

Exemple : nous souhaitions identifier les chaînes de 4 sommets a->b->c->d (donc 3 arêtes) vérifiant une certaine propriété définie par une séquence de fonctions. Le processus sera le suivant :
<br> 1. Initialiser l'état sur le chemin.
<br> 2. Mettre à jour l'état en fonction du sommet a.
<br> 3. Mettre à jour l'état en fonction du sommet b.
<br> 4. Mettre à jour l'état en fonction du sommet c.
<br> 5. Et enfin, mettre à jour l'état en fonction du sommet d.

Si l'état final correspond à nos conditions, alors le filtre accepte la chaîne.

Identifier les chaînes de 4 sommets où au moins 2 des 3 arêtes sont des relations "friend". On suivra l'état suivant : nombre actuel d'arêtes "friend".
Ne pas oublier qu'il est possible d'utiliser les fonctions de F (functions) importé en début de notebook.

In [ ]:
# Votre code ici


## Sous-graphes
GraphFrames fournit une API pour construire des sous-graphes en filtrant sur les arêtes et les sommets.
<br>A partir de notre graphe complet, construire le graphe n'incluant que les personnes de plus de 30 ans et qui ont des amis de plus de 30 ans.
<br>S'aider de la documentation pour filtrer les sommets et les arêtes. Indication : il existe une méthode pour supprimer les Objets isolés une fois les sommets et arêtes filtrés.

In [ ]:
# Votre code ici


## Algorithmes de graphes classiques

GraphFrames fournit un certain nombre d'algorithmes "built-in", dont les plus notables sont :
* Breadth-first search (BFS)
* Connected components
* Strongly connected components
* Label Propagation Algorithm (LPA)
* PageRank (classique et personnalisé)
* Shortest paths
* Triangle count

Dans cette formation, nous nous intéresserons à PageRank et Shortest paths.

## PageRank

Le but de PageRank est d'identifier les sommets "importants".

De votre point de vue, quel sommet du graphe est le plus important ?
<br>Lancer l'algorithme PageRank sur notre graphe avec les paramètres suivants :
<br>resetProbability=0.15
<br>tol=0.01
<br>Afficher les résultats.

In [ ]:
# Votre code ici

Contrairement au PageRank standard, qui calcule les scores de pertinence en fonction de la structure globale du graphe, le PageRank personnalisé prend en compte les préférences ou les priorités spécifiques de l'utilisateur.
<br> Dans ce contexte de PageRank personnalisé, sourceId permet de régler le noeud (via son identifiant) à partir duquel l'algorithme commence à évaluer la pertinence des autres nœuds du graphe.
<br> Relancer l'algorithme PageRank en choisissant le noeud "f" correspondant à Fanny comme sourceId.

In [ ]:
# Votre code ici

Essayer d'expliquer le résultat.
A votre avis, quelle(s) propriété(s) de graphe(s) rendent le point de départ d'autant plus important ?


## Shortest paths

Calcule les chemins les plus courts vers un ensemble donné de sommets "repères" (landmarks en anglais).

A l'aide de l'algorithme shortestPaths, calculer , pour chaque sommet, la longueur du plus court chemin reliant ce sommet au sommet "a" et celle du plus court chemin reliant ce sommet au sommet "d" (s'il existe de tels chemins).

In [ ]:
# Votre code ici

Bonus : Choisir un algorithme supplémentaire dans la liste des algorithmes classiques et le tester !

In [ ]:
# Votre code ici